Upload files to colab

Translation function

In [0]:
#!/usr/bin/env python
# coding: utf-8

# Translate sentences to English and save mapping dicitionary as json.

# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

# Parameters:
# *   sentences =
# *   dictpath =  json dictionary file mapping original sentence with translation
# 
# 

# In[ ]:


from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'pt-br')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        try:
            translations[item]
        except KeyError:
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Prepare input

In [22]:
from pprint import pprint
import pandas as pd
import os

sentences = list()
label_idx = [
             'sentence',
            ]
length = 0

for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size


assert length == len(sentences)
sentences = set(sentences)

train.tsv
0             hide new secretions from the parental units 
1                     contains no wit , only labored gags 
2        that loves its characters and communicates som...
3        remains utterly satisfied to remain the same t...
4        on the worst revenge-of-the-nerds clichés the ...
                               ...                        
67344                                 a delightful comedy 
67345                     anguish , anger and frustration 
67346    at achieving the modest , crowd-pleasing goals...
67347                                    a patient viewer 
67348    this new jangle of noise , mayhem and stupidit...
Name: sentence, Length: 67349, dtype: object



test.tsv
0                  uneasy mishmash of styles and genres .
1       this film 's relationship to actual tension is...
2       by the end of no such thing the audience , lik...
3       director rob marshall went out gunning to make...
4       lathan and diggs have considerable personal 

In [0]:
# erro 'artificial suspense '
#'suspense artificial'
#sentences.remove('artificial suspense ')

#erro 'portuguese master manoel de oliviera '
#'mestre português manoel de oliviera'
#sentences.remove('portuguese master manoel de oliviera ')

#erro  'pedro '
#'pedro '
#sentences.remove('pedro ')

#erro 'alexandre '
#'alexandre '
#sentences.remove('alexandre ')

#erro 'superficial humor '
#'humor superficial'
#sentences.remove('superficial humor ')

#erro 'café '
#'café '
#sentences.remove('café ')

#erro 'a marvel '
#'uma maravilha'
#sentences.remove('a marvel ')

#erro 'wheezy '
#'chiado'
#sentences.remove('wheezy ')

In [0]:
import json
with open('sst-2.json') as reader:
    translation = json.load(reader)

translation['artificial suspense '] = 'artificial suspense '
translation['portuguese master manoel de oliviera '] = 'portuguese master manoel de oliviera '
translation['pedro '] = 'pedro '
translation['alexandre '] = 'alexandre '
translation['superficial humor '] = 'humor superficial'
translation['café '] =  'café '
translation['a marvel '] = 'uma maravilha'
translation['wheezy '] = 'chiado'

with open('sst-2.json', 'w') as writer:
    json.dump(translation, writer)

Translate

In [0]:
translate2dict(sentences,  'sst-2.json')

Replace with translation

In [0]:
from os import mkdir
import json

with open('sst-2.json') as reader:
    dictionary = json.load(reader)

mkdir('translation')
for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, sep = '\t')